In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

# 10-7. 번역기 만들기 (1) 데이터 전처리

### 파일 읽어오기

In [2]:
file_path = "./data/fra.txt"
lines = pd.read_csv(file_path,names=['eng','fra','cc'],sep='\t')
print("전체 샘플의 수 : ",len(lines))
lines.sample(5)

전체 샘플의 수 :  197463


,eng,fra,cc
196245,An 18-year-old male is in hospital after an al...,Un jeune homme de dix-huit ans est à l'hôpital...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
67766,Tom watched impatiently.,Tom a regardé avec impatience.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
187868,How many different pieces are there in Japanes...,Combien de pièces différentes comportent les é...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
41357,Everyone else waited.,Toutes les autres ont attendu.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
163684,The old man has lived here all his life.,Le vieil homme a vécu ici toute sa vie.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


### 불필요한 세번째 열 제거 & 5만개의 샘플만 사용

In [3]:
lines = lines[['eng','fra']][:50000]
lines.sample(5)

,eng,fra
43501,I'm a French teacher.,Je suis enseignant en français.
42733,I like her very much.,Je l'apprécie beaucoup.
22268,You were seventh.,Tu as été septième.
33846,You look satisfied.,Vous avez l'air satisfait.
19003,I never told Tom.,Je ne l’ai jamais dit à Tom.


### seq2seq 동작을 위해 디코더의 입력과 예측에는 시작토큰과 종료토큰이 필요

In [4]:
sos_token = "\t"
eos_token = "\n"
lines.fra = lines.fra.apply(lambda x : '\t' + x + '\n')
print("전체 샘플의 수 : ",len(lines))
lines.sample(5)

전체 샘플의 수 :  50000


,eng,fra
9640,Are you lonely?,\tEs-tu esseulé ?\n
41436,Give me your address.,\tDonnez-moi votre adresse !\n
2854,Cook for me.,\tCuisinez pour moi.\n
26804,Tom misunderstood.,\tTom a mal compris.\n
12318,Tom stopped it.,\tTom l'a arrêté.\n


### 각 단어에 부여된 정수로 텍스트를 정수 인코딩 해주기 

In [6]:
eng_tokenizer = Tokenizer(char_level=True)
eng_tokenizer.fit_on_texts(lines.eng) #토큰화 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng) #단어를 숫자 인덱스로 변경
input_text[:3]

[[19, 3, 8], [19, 3, 8], [19, 3, 8]]

In [7]:
fra_tokenizer = Tokenizer(char_level=True)
fra_tokenizer.fit_on_texts(lines.fra) #토큰화 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra) #단어를 숫자 인덱스로 변경
target_text[:3]

[[10, 19, 5, 1, 31, 11],
 [10, 15, 5, 12, 16, 29, 2, 14, 11],
 [10, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 11]]

### 단어장의 크기를 변수로 저장해주기

In [9]:
eng_vocab_size = len(eng_tokenizer.word_index)+1
fra_vocab_size = len(fra_tokenizer.word_index)+1
print(eng_vocab_size,fra_vocab_size)

53 73


### 데이터의 최대 길이를 구하기 -> 패딩의 크기를 설정하기 위해

In [12]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print(max_eng_seq_len,max_fra_seq_len)

22 74


### 통계정보를 확인해보기 

In [13]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 53
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 22
프랑스어 시퀀스의 최대 길이 74


### target sequence를 두가지로 만들어주기

In [15]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [16]:
print(decoder_input[:3])
print(decoder_target[:3])

[[10, 19, 5, 1, 31], [10, 15, 5, 12, 16, 29, 2, 14], [10, 2, 7, 1, 12, 9, 8, 4, 2, 1, 31]]
[[19, 5, 1, 31, 11], [15, 5, 12, 16, 29, 2, 14, 11], [2, 7, 1, 12, 9, 8, 4, 2, 1, 31, 11]]


### 인코더와 디코더에 패딩 넣어주기 

In [17]:
encoder_input = pad_sequences(encoder_input,maxlen=max_eng_seq_len,padding='post')
decoder_input = pad_sequences(decoder_input,maxlen=max_fra_seq_len,padding='post')
decoder_target = pad_sequences(decoder_target,maxlen=max_fra_seq_len,padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22)
프랑스어 입력데이터의 크기(shape) : (50000, 74)
프랑스어 출력데이터의 크기(shape) : (50000, 74)


### 정수 인코딩된 벡터들에 대해 벡터화 방법으로 원-핫 인코딩을 해주기 

In [18]:
encoder_input = to_categorical(encoder_input)
decoder_input=  to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22, 53)
프랑스어 입력데이터의 크기(shape) : (50000, 74, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 74, 73)


### validation을 위한 데이터셋 나누기 

In [23]:
n_of_val = 3000

encoder_input_val = encoder_input[:n_of_val]
decoder_input_val = decoder_input[:n_of_val]
decoder_target_val = decoder_target[:n_of_val]

encoder_input_train = encoder_input[n_of_val:]
decoder_input_train = decoder_input[n_of_val:]
decoder_target_train = decoder_target[n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_val))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_val))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_val))


영어 학습데이터의 크기(shape) : (3000, 22, 53)
프랑스어 학습 입력데이터의 크기(shape) : (3000, 74, 73)
프랑스어 학습 출력데이터의 크기(shape) : (3000, 74, 73)


# 10-7. 번역기 만들기 (2) 모델 훈련하기

In [24]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# 10-7. 번역기 만들기 (3) 모델 테스트하기